In [1]:
import json

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = ["unsloth/llama-3-8b-bnb-4bit"]

train_instruction = "SUMMARIZE THIS SEARCH"
instruction_title = "DESCRIBE THIS AMAZON SEARCH"


DATA_PATH = "data/trn.json"
OUTPUT_PATH_DATASET = "data/trn-format.json"

data = []
count = 0

data_rows = 100 #Alterar numero de linhas do json

with open(DATA_PATH, encoding='utf-8') as rows:
    for i in rows:
        if count < data_rows: 
            data.append(json.loads(i))
            count += 1

def format_dataset_into_model_input(data):

    inputs = []
    outputs = []

    for i in data:
        inputs.append(i['title'])
        outputs.append(i['content'])

    formatted_data = {
        "input": inputs,
        "output": outputs
    }

    title_data = inputs

    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    return title_data

In [2]:
title_data = format_dataset_into_model_input(data)
data = ['']
i = ''

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


d:\Desenvolvedor\FIAP\Codes\fiap-tech-challenge-3\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4060 Ti. Max memory: 7.995 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


d:\Desenvolvedor\FIAP\Codes\fiap-tech-challenge-3\venv\Lib\site-packages\unsloth\models\llama.py:1277: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from datasets import load_dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = train_instruction
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 1 examples [00:00, 124.86 examples/s]
Map: 100%|██████████| 1/1 [00:00<00:00, 124.89 examples/s]


In [6]:
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.
Converting train dataset to ChatML: 100%|██████████| 1/1 [00:00<00:00, 166.51 examples/s]
num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.
Applying chat template to train dataset: 100%|██████████| 1/1 [00:00<00:00, 195.96 examples/s]
num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.
Tokenizing train dataset: 100%|██████████| 1/1 [00:00<00:00, 24.71 examples/s]
num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.
Tokenizing train dataset: 100%|██████████| 1/1 [00:00<00:00, 71.38 examples/s]


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.272100
2,2.272100
3,2.265000
4,2.229800
5,2.147600
6,2.005600
7,1.829500
8,1.696800
9,1.591800
10,1.514600


In [8]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        train_instruction,
        "Girls Ballet Tutu Neon Pink",
        "High quality 3 layer ballet tutu. 12 inches in length",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS SEARCH

### Input:
Girls Ballet Tutu Neon Pink

### Response:
High quality 3 layer ballet tutu. 12 inches in length. Available in 6 colors. 100% Nylon.<|end_of_text|>


In [9]:
model.save_pretrained("data/trn_model")
tokenizer.save_pretrained("data/trn_model")


('data/trn_model\\tokenizer_config.json',
 'data/trn_model\\special_tokens_map.json',
 'data/trn_model\\tokenizer.json')

In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "data/trn_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="cuda:0",
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4060 Ti. Max memory: 7.995 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[alpaca_prompt.format(
    instruction_title,
    title_data[0],
    "",

)], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    DESCRIBE THIS AMAZON SEARCH

    ### Input:
    Girls Ballet Tutu Neon Pink

    ### Response:
    <|end_of_text|>
